In [1]:
import fasttext
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import string
import re
import random
from pattern.en import wordnet, pluralize

from scipy.spatial.distance import cdist, cosine, euclidean
from scipy.stats import ttest_ind, ttest_1samp
from sklearn.decomposition import PCA, FastICA
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

/Users/harinisuresh/.virtualenvs/test_cases/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Returns a list of predictions (prob of moderation) for a list of examples and a given model. 
# allExamples: list of strings 
# model: fastText model object
def getModelPredsHelper(allExamples, model):
    exampleList = [x.replace('\n', ' ') for x in allExamples]
    preds = model.predict(exampleList)
    preds_int = np.array([1 if 'positive' in p[0] else 0 for p in preds[0]])
    preds_prob = np.array([p[0] for p in preds[1]])
    probs = np.array([1 - p if preds_int[i] == 0 else p for (i,p) in enumerate(preds_prob)])
    probs = [np.round(p,3) for p in probs]
    return probs

# vec: query vector (list of floats)
# all_vecs: pool of all vectors from which neighbors are retrieved (list of list of floats)
# comments: comments corresponding to the vectors in all_vecs (list of strings)
# n: number of neighbors to return (int)
# return_idx: if true, returns indices of nearest neighbors instead of the actual comments (bool)
def getKNNFromVector(vec, all_vecs, comments, n=30, return_idx=False):
    dist_vec = cdist(vec, all_vecs, 'cosine')
    top_vec_idx = np.argsort(dist_vec[0])[1:n+1]
    if return_idx: return top_vec_idx
    top_comments = np.array(comments)[top_vec_idx]
    top_comments = [c for c in top_comments]
    return top_comments

In [10]:
ls ../react-app/api/data/reddit/funny

comments.tsv   embs.npy       embs.tsv       projection_df


In [13]:
DATA_FPATH = '../react-app/api/data/reddit/funny/projection_df'
MODEL_FPATH = '../react-app/api/models/reddit/funny_model.bin'
USE_PATH = '/Users/harinisuresh/Dev/tf_hub/universal-sentence-encoder_4/' # replace with your path

In [5]:
subreddits = ['askscience', 'conspiracy', 'funny', 'hillaryclinton', 'history']

## Load Data

In [14]:
all_comments = pd.read_csv(DATA_FPATH)
embed = hub.load(USE_PATH);
all_comments.head()

Unnamed: 0  comment_id                                            comment  \
0           0           0  You can tell this is fake because it shows a b...   
1           1           1                           The fat lady is singing.   
2           2           2  Good thing volks never break down. XD Shitty k...   
3           3           3                I'm dreaming of a bright christmas.   
4           4           4  I'd be sad too if my mom got knocked up in hig...   

   label   pred    umap_x    umap_y     pca_x     pca_y     tsne_x     tsne_y  
0      1  0.988 -1.342861  8.528571 -0.143643  0.085669  55.842620  13.659914  
1      1  0.997  3.544253  7.443844 -0.008068  0.004616   9.396464  18.984072  
2      1  0.990  0.164953  8.376058 -0.106509  0.059272  26.409739 -35.093937  
3      1  0.995  0.210002  7.736676  0.071565  0.069409 -11.741515  32.897507  
4      1  0.961  3.808352  7.543086 -0.131564  0.022218  13.745846  -3.450827

In [16]:
sub = 'funny'
comments = all_comments.comment
labels = all_comments.label
vecs = np.array(embed(comments))

In [17]:
subreddit_model = fasttext.load_model(MODEL_FPATH)

# Functions for finding synonyms and antonyms 

(from https://github.com/marcotcr/checklist/blob/master/checklist/text_generation.py)

In [18]:
def all_synsets(word, pos=None):
    map = {
        'NOUN': wordnet.NOUN,
        'VERB': wordnet.VERB,
        'ADJ': wordnet.ADJECTIVE,
        'ADV': wordnet.ADVERB
        }
    if pos is None:
        pos_list = [wordnet.VERB, wordnet.ADJECTIVE, wordnet.NOUN, wordnet.ADVERB]
    else:
        pos_list = [map[pos]]
    ret = []
    for pos in pos_list:
        ret.extend(wordnet.synsets(word, pos=pos))
    return ret

def clean_senses(synsets):
    return [x for x in set(synsets) if '_' not in x]
    
def all_possible_synonyms(word, pos=None):
    ret = []
    for syn in all_synsets(word, pos=pos):
        ret.extend(syn.senses)
    return clean_senses(ret)

def all_possible_antonyms(word, pos=None):
    ret = []
    for syn in all_synsets(word, pos=pos):
        if not syn.antonym:
            continue
        for s in syn.antonym:
            ret.extend(s.senses)
    return clean_senses(ret)

def all_possible_hypernyms(word, pos=None, depth=None):
    ret = []
    for syn in all_synsets(word, pos=pos):
        ret.extend([y for x in syn.hypernyms(recursive=True, depth=depth) for y in x.senses])
    return clean_senses(ret)
    
def all_possible_hyponyms(word, pos=None, depth=None):
    ret = []
    for syn in all_synsets(word, pos=pos):
        ret.extend([y for x in syn.hyponyms(recursive=True, depth=depth) for y in x.senses])
    return clean_senses(ret)

## Getting names, cities, and countries

`names.json` and `basic.json` comes from https://github.com/marcotcr/checklist/tree/master/checklist/data. This needs to be downloaded in the `data` folder before this cell is run.

In [22]:
def load_proper_noun_data():
    basic = json.load(open("../react-app/api/proper_noun_data/basic.json"))
    names = json.load(open("../react-app/api/proper_noun_data/names.json"))
    name_set = { x:set([name.lower() for name in names[x]]) for x in names }
    data = {
        'name': names,
        'name_set': name_set,
        'city': basic['city'],
        'city_set': set([city.lower() for city in basic['city']]),
        'country': basic['country'],
        'country_set': set([country.lower() for country in basic['country']]),
    }
    return data

proper_noun_data = load_proper_noun_data()

## Nearest Neighbor Search for Example Set Generation

In [29]:
seed = """
poor snowflake do you need a safe space
"""

comments = np.array(comments)
top_vec_idx = getKNNFromVector(embed([seed]), vecs, comments, n=50, return_idx=True)
knn_vecs = vecs[top_vec_idx]
knn_comments = comments[top_vec_idx]
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(knn_vecs)
cluster_comments_arr = []

for cluster in range(num_clusters):
    cluster_comments = knn_comments[np.where(kmeans.labels_ == cluster)]
    
    print('===================== CLUSTER %i =========================' % cluster)
    for c in cluster_comments:
        print(c, end='\n------------------------\n')
    print("\n\n")
    
    cluster_comments_arr.append(cluster_comments)

===================== CLUSTER 0 =========================
American here, do you want a tub to catch all those snowflake tears in? Look, another way to bring back jobs! Also, free salt for the winter storms!
------------------------
You shouldn't refer to Republicans as snowflakes... they can't handle it.
------------------------
Finally something to decorate my safe space with!
------------------------
I'm sorry if I triggered you, precious delicate snowflake. Please don't smear fake blood on your face and pull the fire alarm. Did I give you PTSD just now? Hug it out with your therapist or your hot-yoga instructor.
------------------------
Your trash female ass is obviously triggered. Go cry in your safe space girl.
------------------------
Should have labeled it NSFW to throw everyone off!
------------------------
Can't decide if you were bullied too much, or not enough. But you got reaaaal sensitive skin, I would recommend a higher spf.
------------------------
Paging r/raisedbynarci

### Next steps: Annie

Example search & generation has three main steps (though they may be intertwined in some ways): 1) getting similar examples, 2) clustering or organizing them in some way 3) visualizing the result.  Right now we're doing 1) with euclidean distance in embedding space of the USE, 2) with k-means clustering, and 3) with just printing out the examples. 

We can think about ways to improve each of these parts.  To start, let's explore 1).  Right now we are getting similar examples with the USE (the model we load from TF Hub).  How does this compare to a different embedding model?  Here are some we can try: 
* BERT: https://huggingface.co/bert-base-uncased
* RoBERTa: https://huggingface.co/roberta-base
* XLNet: https://huggingface.co/xlnet-base-cased

Later, we could also think about fine-tuning some of these embeddings to be better suited to our task/data.  Sample code for loading BERT and getting embeddings for a sample of sentences: 

In [ ]:
# import torch
# from transformers import BertTokenizer, BertModel

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased')
# batch_sentences = ["Hello, my dog is cute", "another sample sentence"]
# tokenized_sentences = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")['input_ids']
# output = bert_model(encoded)['pooler_output'].detach().numpy()
# print(output.shape)

### To do: 
* modify `getKNNFromVector` to take a particular embedding name (e.g., 'BERT') and compute distances in that embedding space. 
* for a handful of seed sentences, get the nearest neighbors and print them out as above in each of the different embedding spaces. Qualitatively note differences you notice among what is returned as similar. Do some seem better or worse?  Are there noticeable differences? 
* for an easy 2 or 3D projection, you can try loading the data into the embedding projector: https://projector.tensorflow.org/  You may want to just do a particular example and its 100 nearest neighbors or something (rather than all the data). You'll have to save the embeddings and sentences as TSV files and then load them in. 

## Testing

In [ ]:
def quantify_shift(shift_target, model, example_list, example_list_2=None, transformations=None, labels = None, labels_2 = None, list_names=["list 1", "list 2"]):
    if transformations:
        example_list_2 = parse_transformation_func(example_list, transformations)

    if shift_target == "predictions":
        preds_1 = getModelPredsHelper(example_list, model)
        preds_2 = getModelPredsHelper(example_list_2, model)
        ttest_result = ttest_ind(preds_1, preds_2)
        if ttest_result.pvalue >= 0.05: 
            print("Predictions are not significantly different.")
        else: 
            operator = "higher" if ttest_result.statistic > 0 else "lower"
            print("P(moderated) for %s is *%s* than for %s (pval = %.3f)" % (list_names[0], operator, list_names[1], ttest_result.pvalue))
            
            
    elif shift_target == "representation":
        vecs_1 = [model.get_sentence_vector(ex) for ex in example_list]
        vecs_2 = [model.get_sentence_vector(ex) for ex in example_list_2]
        intergroup_diffs = cdist(vecs_1, vecs_2).flatten()
        intragroup_diffs = np.concatenate((cdist(vecs_1, vecs_1).flatten(), cdist(vecs_2, vecs_2).flatten()))
        ttest_result = ttest_ind(intergroup_diffs, intragroup_diffs)
        if ttest_result.statistic > 0 and ttest_result.pvalue < 0.05: 
            print("Representations for %s are significantly different from %s (pval = %f)." % (list_names[0], list_names[1], ttest_result.pvalue))
        else: 
            print("Representations for %s and %s are not significantly different." % (list_names[0], list_names[1]))
            
            
    elif shift_target == "performance":
        preds_1 = getModelPredsHelper(example_list, model)
        preds_2 = getModelPredsHelper(example_list_2, model)
        perf_1 = np.array([np.round(preds_1[i]) == labels[i] for i in range(len(preds_1))]).astype(int)
        perf_2 = np.array([np.round(preds_2[i]) == labels_2[i] for i in range(len(preds_2))]).astype(int)
        ttest_result = ttest_ind(perf_1, perf_2)
        if ttest_result.pvalue >= 0.05: 
            print("Model performance on %s and %s is not significantly different." % (list_names[0], list_names[1]))
        else: 
            operator = "higher" if ttest_result.statistic > 0 else "lower"
            print("Model performance on %s is *%s* than for %s (pval = %.3f)" % (list_names[0], operator, list_names[1], ttest_result.pvalue))

In [37]:
# helper to replace words in input
def replace_word(input_list, string_to_replace="", string_to_replace_with=""):
    return [re.sub(string_to_replace, string_to_replace_with, string, flags=re.IGNORECASE) for string in input_list]

# can be used to replace words in input
def replace(input_list, strings_to_replace=None, strings_to_replace_with=None, one_to_one_replace=False):
    if strings_to_replace is None:
        strings_to_replace = []
    if strings_to_replace_with is None:
        strings_to_replace_with = []
    if len(strings_to_replace) != len(strings_to_replace_with) and one_to_one_replace:
        raise RuntimeError("strings_to_replace and strings_to_replace_with must be the same length when one-to-one replacing")
    returned = []
    for string in input_list:
        modified_string = string
        for i in range(len(strings_to_replace)):
            string_to_replace = strings_to_replace[i]
            if one_to_one_replace:
                modified_string = replace_word([modified_string], string_to_replace, strings_to_replace_with[i])[0]
            else:
                modified_string = replace_word([modified_string], string_to_replace, random.choice(strings_to_replace_with))[0]
        returned.append(modified_string)
    return returned

# helper to add words in input
def add_word(input_list, string_to_add=""):
    if string_to_add == "":
        for _ in range(random.randint(20)):
            string_to_add += random.choice(string.ascii_letters + string.digits + string.punctuation)
    return [string + string_to_add for string in input_list]

# can be used to add positive/negative sentiment to input, default adds random
def add(input_list, strings_to_add=None):
    if strings_to_add is None:
        aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', model_type='bert', aug_max=1, action="insert")
        return aug.augment(input_list)
    returned = []
    for string in input_list:
        returned.extend(add_word([string], random.choice(strings_to_add)))
    return returned

# helper to remove words in input
def delete_word(input_list, string_to_delete=""):
    return ["".join(re.split(string_to_delete, string, flags=re.IGNORECASE)) for string in input_list]

# can be used to remove strings in input
def delete(input_list, strings_to_delete=None):
    if strings_to_delete is None:
        aug = naw.RandomWordAug(aug_max=1)
        return aug.augment(input_list)
    for string_to_delete in strings_to_delete:
        input_list = delete_word(input_list, string_to_delete.lower())
    return input_list

In [12]:
# Typo functions. Generally will only make one typo per item in input_list

# typos by keyboard proximity
def keyboard_typo(input_list):
    aug = nac.KeyboardAug(include_special_char=False, include_numeric=False, include_upper_case=False, aug_char_max=1, aug_word_max=1)
    return aug.augment(input_list)

# typos by visual recognition
def ocr_typo(input_list):
    aug = nac.OcrAug(aug_char_max=1, aug_word_max=1)
    return aug.augment(input_list)

# typo caused by inserting a random character
def insert_typo(input_list):
    aug = nac.RandomCharAug(action="insert", include_numeric=False, include_upper_case=False, aug_char_max=1, aug_word_max=1, spec_char="")
    return aug.augment(input_list)

# typo caused by substituting a random character
def substitute_typo(input_list):
    aug = nac.RandomCharAug(action="substitute", include_numeric=False, include_upper_case=False, aug_char_max=1, aug_word_max=1, spec_char="")
    return aug.augment(input_list)

# typo caused by swapping two random adjacent characters
def swap_typo(input_list):
    aug = nac.RandomCharAug(action="swap", include_numeric=False, include_upper_case=False, aug_char_max=1, aug_word_max=1, spec_char="")
    return aug.augment(input_list)

# typo caused by deleting a random character
def delete_typo(input_list):
    aug = nac.RandomCharAug(action="delete", include_numeric=False, include_upper_case=False, aug_char_max=1, aug_word_max=1, spec_char="")
    return aug.augment(input_list)

In [13]:
def synonym_pattern(input_list, words_to_replace=None):
    if words_to_replace is None:
        words_to_replace = [random.choice(string.split()) for string in input_list]
    for word in words_to_replace:
        synonyms = all_possible_synonyms(word)
        if len(synonyms) > 0:
            input_list = replace(input_list, strings_to_replace=[word], strings_to_replace_with=synonyms)
    return input_list

def antonym_pattern(input_list, words_to_replace=None):
    if words_to_replace is None:
        words_to_replace = [random.choice(string.split()) for string in input_list]
    for word in words_to_replace:
        antonyms = all_possible_antonyms(word)
        if len(antonyms) > 0:
            input_list = replace(input_list, strings_to_replace=[word], strings_to_replace_with=antonyms)
    return input_list

def synonym_nlpaug(input_list):
    aug = naw.SynonymAug(aug_src='wordnet', aug_max=1)
    return aug.augment(input_list)

def antonym_nlpaug(input_list):
    aug = naw.AntonymAug(aug_max=1)
    return aug.augment(input_list)

def hyponym(input_list, words_to_replace=None):
    if words_to_replace is None:
        words_to_replace = [random.choice(string.split()) for string in input_list]
    for word in words_to_replace:
        hyponyms = all_possible_hyponyms(word)
        if len(hyponyms) > 0:
            input_list = replace(input_list, strings_to_replace=[word], strings_to_replace_with=hyponyms)
    return input_list

def hypernym(input_list, words_to_replace=None):
    if words_to_replace is None:
        words_to_replace = [random.choice(string.split()) for string in input_list]
    for word in words_to_replace:
        hypernyms = all_possible_hypernyms(word)
        if len(hypernyms) > 0:
            input_list = replace(input_list, strings_to_replace=[word], strings_to_replace_with=hypernyms)
    return input_list

In [ ]:
def names(input_list):
    returned = []
    for input_string in input_list:
        new_string = input_string
        for word in set(input_string.split()):
            cleaned_word = word.lower().translate(str.maketrans('', '', string.punctuation))
            if cleaned_word in proper_noun_data['name_set']['women']:
                new_string = replace_word([new_string], word, random.choice(proper_noun_data['name']['women']))[0]
            elif cleaned_word in proper_noun_data['name_set']['men']:
                new_string = replace_word([new_string], word, random.choice(proper_noun_data['name']['men']))[0]
            elif cleaned_word in proper_noun_data['name_set']['last']:
                new_string = replace_word([new_string], word, random.choice(proper_noun_data['name']['last']))[0]
        returned.append(new_string)
    return returned

def cities(input_list):
    returned = []
    for input_string in input_list:
        new_string = input_string
        for word in set(input_string.split()):
            cleaned_word = word.lower().translate(str.maketrans('', '', string.punctuation))
            if cleaned_word in proper_noun_data['city_set']:
                new_string = replace_word([new_string], word, random.choice(proper_noun_data['city']))[0]
        returned.append(new_string)
    return returned

def countries(input_list):
    returned = []
    for input_string in input_list:
        new_string = input_string
        for word in set(input_string.split()):
            cleaned_word = word.lower().translate(str.maketrans('', '', string.punctuation))
            if cleaned_word in proper_noun_data['country_set']:
                new_string = replace_word([new_string], word, random.choice(proper_noun_data['country']))[0]
        returned.append(new_string)
    return returned

In [14]:
# define transformation funcs here
def parse_transformation_func(input_list, transformations=[]):
    for transformation in transformations:
        output_list = transformation(input_list)
    return output_list

def build_transformation_func(transformation_type, params=[]):
    if transformation_type == 'replace':
        return lambda l: replace(l, params[0], params[1])
    if transformation_type == 'add':
        return lambda l: add(l, params[0])
    if transformation_type == 'delete':
        return lambda l: delete(l, params[0])
    if transformation_type == 'keyboard_typo':
        return lambda l: keyboard_typo(l)
    if transformation_type == 'ocr_typo':
        return lambda l: ocr_typo(l)
    if transformation_type == 'insert_typo':
        return lambda l: insert_typo(l)
    if transformation_type == 'substitute_typo':
        return lambda l: substitute_typo(l)
    if transformation_type == 'swap_typo':
        return lambda l: swap_typo(l)
    if transformation_type == 'delete_typo':
        return lambda l: delete_typo(l)
    if transformation_type == 'synonym':
        return lambda l: synonym(l, params[0] if len(params) > 0 else None)
    if transformation_type == 'antonym':
        return lambda l: antonym(l, params[0] if len(params) > 0 else None)
    if transformation_type == 'hyponym':
        return lambda l: hyponym(l, params[0] if len(params) > 0 else None)
    if transformation_type == 'hypernym':
        return lambda l: hypernym(l, params[0] if len(params) > 0 else None)
    if transformation_type == 'names':
        return lambda l: names(l)
    if transformation_type == 'cities':
        return lambda l: cities(l)
    if transformation_type == 'countries':
        return lambda l: countries(l)
        

In [ ]:
# hostile comments
example_list = ["Dude, I hate to break it to you, but I think you misread my original comment.",
                "So your commenting on how the terminology is slightly off? Fuck off",
                "When you comment like this, you sound like a fucking asshole. Just letting you know.",
                "Damn bro. I just triggered you, my bad. Pokefags are just silly."
                "stop commenting bullshit on my post. if you don't have anything useful to say, get off my post.....,",
                "You are clearly missing the point when you think you need to explain the obvious to me. Oh to be so privileged.",
                "LOOOOOL. You are so fucking stupid and you don't even know it. /r/iamverysmart",
                "You have no idea what you're talking about.",
                "You sound like a real piece of shit OP.",
                "Are you for real? Am I actually seeing this comment somewhat upvoted?",
                "'I know you are but what am I.' You quoted my entire post? Are you actually retarded?",
                "no offense, but youre kindof being an asshole. this whole thread including your posts is fucking filled with stupidity and assholeness",
                "I feel like you really missed an opportunity to type a comment that makes sense."]

# polite disagreement comments
example_list_2 = [
    "i totally see where your coming from. i don’t think however that we should approach this with a gradient…",
    "I think we're going in a different direction here, but thank you for your contribution."
    "Yeah. That sounds right. Im just talking out of my ass to be honest. Cheers"
    "You are absolutely correct. I was thinking of reddit when I made my comment. I should've articulated that. Thank you for drawing my attention to my omission."
    "I'm aware, but thank you. I'm just saying, if you're going to correct one thing, it might as well be the worst error.",
    "Thats a clear idea on my side as well,,, nice said dear",
    "Thank you for saying this because I needed your comment to understand the joke here, I didn't see it at all.",
    "Oh, I think we're actually agreeing with each other then. I agree it must be hell, was just trying to say it's real nonetheless. Sorry for that misunderstanding!",
    "And here I was thinking this went the other way around; I guess I learned something new today.",
    "Aww man, I was going to make a similar, way less funny version of this joke. Thanks for getting to it first!"
]

In [ ]:
print(synonym_nlpaug(example_list))
print(synonym_pattern(example_list))

print(antonym_nlpaug(example_list))
print(antonym_pattern(example_list))


['Dude, I detest to break it to you, but I think you misread my original comment.', 'So your commenting on how the terminology is slenderly off? Fuck off', 'When you comment like this, you sound like a fucking asshole. Just letting you know.', "Damn bro. I just triggered you, my bad. Pokefags are just silly. stop remark bullshit on my post. if you don ' t have anything useful to say, get off my post. .. ..,", 'You are clearly missing the point when you think you need to explain the obvious to me. Oh to be so privileged.', "LOOOOOL. You are so fucking stupid and you don ' t even know information technology. / r / iamverysmart", "You have no idea what you ' re talking astir.", 'You sound like a real piece of shit OP.', 'Are you for real? Am I actually seeing this comment somewhat upvoted?', "' I know you are but what am I. ' You quoted my entire post? Are you really retarded?", 'no offense, but youre kindof being an asshole. this whole thread including your posts is fucking filled with s

In [25]:
replace(["i like your COMMENT", "here for the drama"], ["comment"], ["drama"])


['i like your drama', 'here for the drama']

In [51]:
quantify_shift("representation", subreddit_model, example_list, example_list_2=example_list_2, list_names = ["'Hostile'", "'Polite Disagreement'"])

Representations for 'Hostile' are significantly different from 'Polite Disagreement' (pval = 0.000003).


In [20]:
transformations = [build_transformation_func("replace", [".", "!"])]
quantify_shift("representation", subreddit_model, example_list, transformations=transformations, list_names = ["'Hostile'", "'Polite Disagreement'"])

Representations for 'Hostile' and 'Polite Disagreement' are not significantly different.


In [21]:
quantify_shift("performance", subreddit_model, example_list, example_list_2=example_list_2, labels = np.ones(len(example_list)), labels_2 = np.zeros(len(example_list_2)), list_names = ["'Hostile'", "'Civil Disagreement'"])

Model performance on 'Hostile' and 'Civil Disagreement' is not significantly different.


In [22]:
transformations = [build_transformation_func("replace", [".", "!"])]
quantify_shift("representation", subreddit_model, example_list, transformations=transformations, list_names = ["'Hostile'", "'Polite Disagreement'"])

Representations for 'Hostile' and 'Polite Disagreement' are not significantly different.


In [23]:
quantify_shift("predictions", subreddit_model, example_list, example_list_2=example_list_2, list_names = ["'Hostile'", "'Polite Disagreement'"])

P(moderated) for 'Hostile' is *higher* than for 'Polite Disagreement' (pval = 0.013)


In [24]:
transformations = [build_transformation_func("replace", [".", "!"])]
quantify_shift("predictions", subreddit_model, example_list, transformations=transformations, list_names = ["'Hostile'", "'Polite Disagreement'"])

Predictions are not significantly different.


In [65]:
[c + " You Suck!!" for c in cluster_comments_arr[0]]

['American here, do you want a tub to catch all those snowflake tears in? Look, another way to bring back jobs! Also, free salt for the winter storms! You Suck!!',
 "You shouldn't refer to Republicans as snowflakes... they can't handle it. You Suck!!",
 'Finally something to decorate my safe space with! You Suck!!',
 "I'm sorry if I triggered you, precious delicate snowflake. Please don't smear fake blood on your face and pull the fire alarm. Did I give you PTSD just now? Hug it out with your therapist or your hot-yoga instructor. You Suck!!",
 'Your trash female ass is obviously triggered. Go cry in your safe space girl. You Suck!!',
 'Should have labeled it NSFW to throw everyone off! You Suck!!',
 "Can't decide if you were bullied too much, or not enough. But you got reaaaal sensitive skin, I would recommend a higher spf. You Suck!!",
 'Paging r/raisedbynarcissists \n\nAlso have have an upvote because u/temalyen is in a safe space and we love them You Suck!!',
 "Looks like the mods 

In [74]:
preds_1 = getModelPredsHelper(cluster_comments_arr[0], subreddit_model)
preds_2 = getModelPredsHelper([c + " I hate you!!!" for c in cluster_comments_arr[0]], subreddit_model) 

In [75]:
preds_1, preds_2

([0.762, 0.009, 0.011, 0.975, 0.551, 0.016, 0.081, 0.007, 0.989, 0.957],
 [0.636, 0.053, 0.017, 0.953, 0.423, 0.025, 0.076, 0.012, 0.92, 0.743])

In [83]:
from scipy.stats import chisquare, ttest_rel, wilcoxon

In [78]:
def get_confusion_matrix_occurences(pre_transform_preds, post_transform_preds):
    cm_arr = [0, 0, 0, 0]
    for i in range(len(pre_transform_preds)):
        if pre_transform_preds[i] >= 0.5: 
            if post_transform_preds[i] >= 0.5: 
                cm_arr[0] += 1
            else: 
                cm_arr[1] += 1
        else: 
            if post_transform_preds[i] >= 0.5: 
                cm_arr[2] += 1
            else: 
                cm_arr[3] += 1
                
    pre_transform_preds = np.array(pre_transform_preds)
    ideal_cm_arr = [len(np.where(pre_transform_preds >= 0.5)[0]), 0, 0, len(np.where(pre_transform_preds < 0.5)[0])]
    
    chisquare(cm_arr, ideal_cm_arr)
    
    return cm_arr, ideal_cm_arr, chisquare(cm_arr, ideal_cm_arr, ddof=1)
        

In [88]:
x = wilcoxon((np.array(preds_1) >= 0.5).astype(int),(np.array(preds_2) >= 0.5).astype(int))

In [95]:
test = dict()
test = WilcoxonResult(statistic=0.0, pvalue=0.31731050786291415)

NameError: name 'WilcoxonResult' is not defined

In [79]:
get_confusion_matrix_occurences(preds_1, preds_2)

/Users/harinisuresh/.virtualenvs/test_cases/lib/python3.7/site-packages/scipy/stats/stats.py:6125: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp
/Users/harinisuresh/.virtualenvs/test_cases/lib/python3.7/site-packages/scipy/stats/stats.py:6125: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp


([4, 1, 0, 5], [5, 0, 0, 5], Power_divergenceResult(statistic=nan, pvalue=nan))